Hi, friends.

Let's study how the creation of simple binary features can impact the generalization of logistic regression when evaluated with several metrics: log-loss, AUC, F1-score, and accuracy. By ten-fold cross-validation, we'll see that these custom features improve the mean AUC across folds. Then, given more training data, all scores tend to improve consistently when evaluated on unseen data. 

You can find the complete code for this project on my GitHub: <br>
https://github.com/Justin-Le/predicting-diabetes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import Seaborn for plotting
# and ignore all warnings
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
plt.style.use('ggplot')

data = pd.read_csv('../input/diabetes.csv')
data.shape
data.head()

First, let's consider pairwise relationships between raw features by inspecting their scatterplots. The FacetGrid function in Seaborn allows us to easily color-code the two classes, which helps us to quickly identify regions of interest for feature engineering.

As an example, the first plot leads us to (naively) suspect that individuals who have an insulin-level greater than or equal to 400 tend to be at risk of diabetes (blue). This clue leads us to a potential custom feature: insulin_geq_400. We'll continue this process of naive inspection for the remaining features, too.

In [ ]:
grid = sns.FacetGrid(data, hue='Outcome')
grid.map(plt.scatter, 'Glucose', 'Insulin')
grid.add_legend()
plt.show()

# High risk: insulin_geq_400

Already, we see some zero-values that may need to be imputed later on. Is it possible for a person to have an insulin-level of zero? Similarly, the plot below suggests that some people have a body-mass index of zero, which we suspect is meaningless.

In [ ]:
grid = sns.FacetGrid(data, hue='Outcome')
grid.map(plt.scatter, 'Glucose', 'BMI')
grid.add_legend()
plt.show()

# High risk: BMI_geq_48

In [ ]:
grid = sns.FacetGrid(data, hue='Outcome')
grid.map(plt.scatter, 'Glucose', 'DiabetesPedigreeFunction')
grid.add_legend()
plt.show()

# High risk: pedigree_geq_1

# From all scatterplots
# High risk: glucose_geq_170

In [ ]:
grid = sns.FacetGrid(data, hue='Outcome')
grid.map(plt.scatter, 'BloodPressure', 'SkinThickness')
grid.add_legend()
plt.show()

# High risk: blood_geq_92

Perhaps histograms would be more appropriate for examining these next few features, as they each have only a few discrete values.

In [ ]:
n_bins = data.Pregnancies.max() - data.Pregnancies.min()
grid = sns.FacetGrid(data, row='Outcome')
grid.map(plt.hist, 'Pregnancies', bins=np.arange(0, n_bins)-0.5)
plt.show()

# Low risk: preg_1_or_2

As mentioned in the forums by our friend Hugues Fontenelle, it's certainly reasonable for an individual to have zero pregnancies, so we won't impute this value.

In [ ]:
n_bins = data.Age.max() - data.Age.min()
grid = sns.FacetGrid(data, row='Outcome')
grid.map(plt.hist, 'Age', bins=np.arange(20, n_bins)-0.5)
plt.show()

# Low risk: age_leq_28
# High risk: age_52_or_53

In [ ]:
data = data[data.SkinThickness != 0]
n_bins = data.SkinThickness.max() - data.SkinThickness.min()
grid = sns.FacetGrid(data, row='Outcome')
grid.map(plt.hist, 'SkinThickness', bins=np.arange(n_bins)-0.5)
plt.show()

# Low risk: st_10_to_23

Before we test the custom features that were suggested from these plots, let's see how logistic regression and XGBoost perform when cross-validated on the raw data. Then, we'll see how they generalize to the isolated testing set when given the complete training set to learn from.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, f1_score, accuracy_score

X = data.ix[:, :-1]
y = data.ix[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Instantiate logistic regression
clf = LogisticRegression()

#Cross-validate logistic regression
print("\nLogistic regression results for 10-fold cross-validation:\n")
scores = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
print(("Accuracies:\n %s\n\n" +
       "Best accuracy on held-out data: %.4f\n\n" +
       "Mean accuracy on held-out data: %.4f\n\n") % (str(scores), scores.max(), scores.mean()))
scores = cross_val_score(clf, X_train, y_train, cv=10, scoring='roc_auc')
print(("AUC:\n %s\n\n" +
       "Best AUC on held-out data: %.4f\n\n" + 
       "Mean AUC on held-out data: %.4f\n\n") % (str(scores), scores.max(), scores.mean()))

clf.fit(X_train, y_train)
pred = clf.predict(X_test)

print("="*80)
print("\nLogistic regression performance on unseen data:")
print("\nlog-loss: %.4f" % log_loss(y_test, pred))
print("\nAUC: %.4f" % roc_auc_score(y_test, pred))
print("\nF1 score: %.4f" % f1_score(y_test, pred))
print("\nAccuracy: %.4f" % accuracy_score(y_test, pred))
print("="*80)

Let's see if we can improve these scores by creating new features and imputing some bad ones.

In [ ]:
######################################## 
# Create binary features
######################################## 

# Insulin >= 400
X['insulin_geq_400'] = np.where(X['Insulin'] >= 400, 1, 0)

# BMI >= 48
X['bmi_geq_48'] = np.where(X['BMI'] >= 48, 1, 0)

# Diabetes Pedigree Function >= 1
X['pedigree_geq_1'] = np.where(X['DiabetesPedigreeFunction'] >= 1.0, 1, 0)

# Glucose >= 170
X['glucose_geq_170'] = np.where(X['Glucose'] >= 170, 1, 0)

# Blood Pressure >= 92
X['blood_geq_92'] = np.where(X['BloodPressure'] >= 92, 1, 0)

# One or two pregnancies
X['preg_1_or_2'] = np.where(X['Pregnancies'] == 1, 1, 0) +\
                      np.where(X['Pregnancies'] == 2, 1, 0)

# Age <= 28
X['age_leq_28'] = np.where(X['Age'] <= 28, 1, 0)

# Age is 52 or 53
X['age_52_or_53'] = np.where(X['Age'] == 52, 1, 0) +\
                       np.where(X['Age'] == 53, 1, 0)

# 10 <= Skin Thickness >= 23
X['skin_10_to_23'] = np.where(X['SkinThickness'] >= 10, 1, 0) -\
                        np.where(X['SkinThickness'] > 23, 1, 0)

X.head()

Certainly, there's potential for more intricate features, but we'll only consider these naive binary ones for now.

Before imputing, let's isolate training and testing sets.

In [ ]:
from sklearn.model_selection import train_test_split

# Isolate a training set for CV.
# Testing set won't be touched until CV is complete.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Note that the train/test split *must* be done before any imputation.
# If we impute values before isolating the training set,
# then values from the testing set might 
# influence the imputed values in the training set.
# See Abu-Mostafa's "Learning from Data" to read more about this issue,
# sometimes referred to as "data snooping".

We'll now impute the meaningless zeros in the dataset by replacing them with random Gaussian values localized near the respective sample means. Again, this step must be done separately for training and testing sets, and we'll see why in a moment.

In [ ]:
normals = [0]*3
variables = ['Glucose', 'SkinThickness', 'BMI']

# Generate imputation values with Gaussian randomness.
for n, v in zip(range(len(normals)), variables):
    # Shift the mean up to account for skewness caused by zeros.
    v_mean = X_train[v].mean()*1.5

    # Use surrogate deviation.
    # (Sometimes I get strange values when using .std(). Why?)
    v_std = v_mean*0.1

    normals[n] = np.random.normal(loc = v_mean, scale = v_std)

print("Imputing zeros in Glucose, SkinThickness, and BMI with")
print("%f, %f, and %f" % (normals[0], normals[1], normals[2]))

# Impute.
X_train = X_train.replace(to_replace = {'Glucose': {0: normals[0]}, 
                                  'SkinThickness': {0: normals[1]}, 
                                  'BMI': {0: normals[2]}})

In [ ]:
normals = [0]*3
variables = ['Glucose', 'SkinThickness', 'BMI']

# Generate imputation values with Gaussian randomness.
for n, v in zip(range(len(normals)), variables):
    # Shift the mean up to account for skewness caused by zeros.
    v_mean = X_test[v].mean()*1.5

    # Use surrogate deviation.
    # (Sometimes I get strange values when using .std(). Why?)
    v_std = v_mean*0.1

    normals[n] = np.random.normal(loc = v_mean, scale = v_std)

print("Imputing zeros in Glucose, SkinThickness, and BMI with")
print("%f, %f, and %f" % (normals[0], normals[1], normals[2]))

# Impute.
X_test = X_test.replace(to_replace = {'Glucose': {0: normals[0]}, 
                                  'SkinThickness': {0: normals[1]}, 
                                  'BMI': {0: normals[2]}})

Notice how the imputing values are very different in the training set than they are in the testing set. If we had imputed before the train/test split, then information from the testing set would have significantly influenced our imputation of the training set. Let's try to avoid this kind of information leakage, as it could hinder generalization. For more on this issue, please see Abu-Mostafa's "Learning from Data", an excellent book that formally discusses generalization in the theory of statistical learning.

Also notice that we haven't imputed insulin-levels for now, since there are too many observations that would be affected, and we'd possibly require a more sophisticated imputation strategy to avoid causing a significant skew in the sample distribution.

To see the impact of the custom features, we'll first cross-validate using only the training data.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, f1_score, accuracy_score

# Instantiate logistic regression
clf = LogisticRegression()

#Cross-validate logistic regression
print("\nLogistic regression results for 10-fold cross-validation:\n")
scores = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
print(("Accuracies:\n %s\n\n" +
       "Best accuracy on held-out data: %.4f\n\n" +
       "Mean accuracy on held-out data: %.4f\n\n") % (str(scores), scores.max(), scores.mean()))
scores = cross_val_score(clf, X_train, y_train, cv=10, scoring='roc_auc')
print(("AUC:\n %s\n\n" +
       "Best AUC on held-out data: %.4f\n\n" + 
       "Mean AUC on held-out data: %.4f\n\n") % (str(scores), scores.max(), scores.mean()))

Compare the above results to the performance on raw data. The mean AUC increased from 81 to 84%, while the peak scores fell slightly.

Now, we'll see how these features help the model to generalize to the isolated testing set after being given access to the complete training set.

In [ ]:
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

print("="*80)
print("\nLogistic regression performance on unseen data:")
print("\nlog-loss: %.4f" % log_loss(y_test, pred))
print("\nAUC: %.4f" % roc_auc_score(y_test, pred))
print("\nF1 score: %.4f" % f1_score(y_test, pred))
print("\nAccuracy: %.4f" % accuracy_score(y_test, pred))
print("="*80)

 - Log-loss decreased from 8.24 to 6.65.
 - AUC increased from 70 to 75%.
 - F1-score increased from 59 to 67%.
 - Accuracy increased from 76 to 80%.

All of the scores improved for this particular train/test split.

We can change the random seed of the split and see if this performance is consistent with other random splits. (The results are similar with seeds 5 and 10.)

On that note, I'd like to give a big thanks to Hugues Fontenelle for reminding all of us about the "luckiness" of splits. Be sure to check out his notebook, as well.

Best wishes, everyone.